In [1]:
import requests, json
from bs4 import BeautifulSoup
from datetime import datetime

In [2]:
# The beginning and end time of daily observation.
obs_begin = '12'
obs_stop = '20'

# Scrawling _[NEO Earth Close Approaches](https://cneos.jpl.nasa.gov/ca/)_ !
* Table Settings: Nominal dist. <= __0.2__au

<img src="https://drive.google.com/uc?id=11HAL3jkXzH52n9MYPNCBbi2HISN8rtRk", width=1000, height=1000>

In [3]:
res = requests.get("https://ssd-api.jpl.nasa.gov/cad.api?www=1&nea-comet=Y&dist-max=0.2&fullname=true")
soup = BeautifulSoup(res.text, 'lxml')

# All data for each object.
jd = json.loads(soup.text)
jd['data'][0]

['bK19G04G|2019 GG4',
 '3',
 '2458605.655835687',
 '2019-May-02 03:44',
 '0.0778433095991995',
 '0.0775389731372091',
 '0.0781476434719006',
 '7.8962756748613',
 '7.89193968739273',
 '00:01',
 '24.951',
 '       (2019 GG4)']

# Choosing today's NEAs by __Close-Approach (CA) Date__

In [4]:
today = datetime.now().strftime("%Y-%b-%d")
neas = [i[0].split('|')[1] for i in jd['data'] if today in i[3]]
neas

['2019 GG4', '2017 SF2', '2012 CP2']

# Scrawling _[Minor Planet & Comet Ephemeris Service](https://www.minorplanetcenter.net/iau/MPEph/MPEph.html)_
* __neaBright__: Observable NEAs
* __neaMotions__: NEA's motion ["/s]
* __neaLines__: NEA's ephemerides (ACP's format for NEOCP)

In [5]:
dateWP = datetime.now().strftime("%Y %m %d")
date = datetime.now().strftime("%Y%m%d")
neaMotions, neaLines, neaBright = [], [], []
for nea in neas:
    neaLine = ""
    payload = {'ty':'e', 'TextArea':nea, 'd':dateWP, 'l':'', 'i':'1', 'u':'h', 'uto':'0', 'c':'D37'
               , 'long':'', 'lat':'', 'alt':'', 'raty':'d', 's':'t', 'm':'s', 'igd':'y', 'ibh':'y', 'adir':'N'
               , 'oed':'', 'e':'-2', 'resoc':'', 'tit':'', 'bu':'', 'ch':'c', 'ce':'f', 'js':'f'}
    res = requests.post("https://cgi.minorplanetcenter.net/cgi-bin/mpeph2.cgi", data = payload)
    soup = BeautifulSoup(res.text, 'lxml')

    ephems = [i for i in soup.select('pre')[0].text.split('\n') if dateWP in i]
    ephemsOK = [i for i in ephems if (int(i.split()[14][1:]) >= 40) and (i.split(' ')[3][:2] >= obs_begin) and (i.split(' ')[3][:2] <= obs_stop)]
    # NEA is observable and its magnitude is brighter than 18-mag!
    if (ephemsOK != []) and (float(ephemsOK[0].split()[10]) <= 18.):
        neaMotions.append(float(ephemsOK[0].split()[11]))
        for ephem in ephemsOK:
    #         with open("D:/LWTdata/LWT_{}/lulinLWT/others/{}_ephem_{}.txt".format(date, nea, ephemsOK[0].split()[11]), 'a') as file:
    #             file.write(ephem + '\n')
            neaLine = neaLine + ephem[:36] + '|'
        neaLines.append(neaLine[:-1])
        neaBright.append(nea)
    else:
        pass

In [6]:
print(neaBright)
print(neaMotions)
print(neaLines)

[]
[]
[]


# Choosing one which has the __longest__ observable time span!
## If no NEOs and NEAs, send an email!

In [7]:
if neaBright != []:
    neaSpans = [(int(line.split('|')[-1].split()[3][:2])-int(line.split('|')[0].split()[3][:2])) for line in neaLines]
    neaMaxSpanIdx = neaSpans.index(max(neaSpans))
    neaBOK = neaBright[neaMaxSpanIdx]
    neaLOK = neaLines[neaMaxSpanIdx]
    neaMOK = neaMotions[neaMaxSpanIdx]
else:
#     shutil.rmtree("D:/LWTdata/LWT_{}/lulinLWT".format(date))
#     sendemail(from_addr    = 'lwt@gm.astro.ncu.edu.tw', 
#               to_addr_list = ['lwtgroup@astro.ncu.edu.tw'],
#               cc_addr_list = [], 
#               subject      = 'LWT has "NO" observation today!', 
#               message      = 'Bonjour,\n\nPlease run the script of Prof. Ngeow today!\nMerci beaucoup!\n\nAmuse-toi bien,\nJian-Fong Huang (smoBEE)\nemail: smoBEE@astro.ncu.edu.tw', 
#               login        = 'lwt@gm.astro.ncu.edu.tw', 
#               password     = '')
#     sys.exit()
    pass

In [8]:
print('[{}] motion is: {}"/s'.format(neaBOK, neaMOK))
print('[{}] ephemerise is:\n{}'.format(neaBOK, neaLOK))

NameError: name 'neaBOK' is not defined